In [1]:
import sys
sys.path.append("/Users/abcom/Desktop/github/reviewturtl")
from reviewturtl.src.agents.summarizer_agent import SummarizerAgent
from reviewturtl.src.agents.code_search_agent import CodeSearchAgent

/Users/abcom/.pyenv/versions/3.11.0/envs/reviewturtl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from reviewturtl.settings import initialize_dspy_with_configs
initialize_dspy_with_configs()

In [3]:
import dspy

In [19]:
agents = [SummarizerAgent, CodeSearchAgent]
agents_dict = {}
arg_agent_mapper = {}
for agent in agents:
    agent_init = agent()
    agents_dict[agent_init.class_name] = agent_init.desc
    arg_agent_mapper[agent_init.class_name] = agent_init.input_variables

In [20]:
agents_dict

{'SummarizerAgent': 'A summarizer agent that summarizes the changes in a file diff.',
 'CodeSearchAgent': 'A code search agent that searches for code in a repository.'}

In [9]:
from typing import List, Dict
class ReactSignature(dspy.Signature):
    __doc__ = """
    INSTRUCTIONS TO FOLLOW\n
    >>> Available agents are the agents that are available to answer the query. They are in a dictionary format with the key as the agent name and the value as the agent description\n
    >>> Think step by step on what agents are best suited to answer the query\n
    >>> Choose the best agents (can be more than one) to answer the query\n
    >>> Return the agents in a list\n
    """
    conversation_history: str = dspy.InputField(desc="The conversation history so far")
    query: str = dspy.InputField(
        desc="The latest user query based on which the agent is to be chosen"
    )
    available_agents: Dict[str, str] = dspy.InputField(
        desc="The Dictionary of available agents with the agent name as the key and the agent description as the value"
    )
    agents_to_use: List[str] = dspy.OutputField(desc="The List of agents to use")

In [10]:
pg = dspy.TypedChainOfThought(ReactSignature)
pred = pg(conversation_history="", query="i want to summarize the changes in this file", available_agents=agents_dict)

In [11]:
pred

Prediction(
    reasoning='{\n  "value": [\n    "SummarizerAgent"\n  ]\n}',
    agents_to_use=['SummarizerAgent']
)

In [21]:
agent_being_used = pred.agents_to_use[0]
args_to_find = arg_agent_mapper[agent_being_used]

In [22]:
args_to_find

['file_diff']

In [ ]:
agent_being_used

In [23]:
file_diff = """
diff --git a/reviewturtl/src/agents/summarizer_agent.py b/reviewturtl/src/agents/summarizer_agent.py
index 83b2c9d..f3b2c9d 100644
--- a/reviewturtl/src/agents/summarizer_agent.py
+++ b/reviewturtl/src/agents/summarizer_agent.py
@@ -1,6 +1,7 @@
 from reviewturtl.src.programmes.typed_cot import DspyProgramme
 from reviewturtl.src.signatures.signatures import SummarizerSignature
 from reviewturtl.src.agents.base_agent import Agent
+import logging
 
 
 class SummarizerAgent(Agent):
@@ -8,6 +9,8 @@ class SummarizerAgent(Agent):
 
     def forward(self, file_diff):
         self.prediction_object = self.programme.predictor(file_diff=file_diff)
+        logging.info("Prediction object created")
         return self.prediction_object
 
     def walkthrough(self):
@@ -20,6 +23,7 @@ class SummarizerAgent(Agent):
 
     def __call__(self, file_diff):
         return self.forward(file_diff)
+        logging.info("SummarizerAgent called with file_diff")
 
 
 __all__ = ["SummarizerAgent"]

"""

In [28]:
class FindArgumentsForAgent(dspy.Signature):
    __doc__ = """
    INSTRUCTIONS TO FOLLOW\n
    >>> 
    """
    context: Dict[str, str] = dspy.InputField(desc="The context whatever is provided to be used for finding the arguments")
    agent_being_used_with_desc: Dict[str, str] = dspy.InputField(desc="The agent being used with the agent name and the agent description")
    argumets_to_find: List[str] = dspy.InputField(desc="The arguments to find for the agent")
    args_with_values: Dict[str, str] = dspy.OutputField(desc="The mapping of the arguments to the agent with key as the argument name and value as the argument value")

In [32]:
args_to_find

['file_diff']

In [33]:
pg = dspy.TypedChainOfThought(FindArgumentsForAgent)
pred = pg(context={'file_diff': file_diff,'commit_hash':"2034","query":"i want to summarize the changes in this file"}, agent_being_used_with_desc={agents_dict[agent_being_used]:agent_being_used}, argumets_to_find=args_to_find)
pred

Prediction(
    reasoning='{\n  "value": {\n    "file_diff": "\\ndiff --git a/reviewturtl/src/agents/summarizer_agent.py b/reviewturtl/src/agents/summarizer_agent.py\\nindex 83b2c9d..f3b2c9d 100644\\n--- a/reviewturtl/src/agents/summarizer_agent.py\\n+++ b/reviewturtl/src/agents/summarizer_agent.py\\n@@ -1,6 +1,7 @@\\n from reviewturtl.src.programmes.typed_cot import DspyProgramme\\n from reviewturtl.src.signatures.signatures import SummarizerSignature\\n from reviewturtl.src.agents.base_agent import Agent\\n+import logging\\n \\n \\n class SummarizerAgent(Agent):\\n@@ -8,6 +9,8 @@ class SummarizerAgent(Agent):\\n \\n     def forward(self, file_diff):\\n         self.prediction_object = self.programme.predictor(file_diff=file_diff)\\n+        logging.info(\\"Prediction object created\\")\\n         return self.prediction_object\\n \\n     def walkthrough(self):\\n@@ -20,6 +23,7 @@ class SummarizerAgent(Agent):\\n \\n     def __call__(self, file_diff):\\n         return self.forward(fil